In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


aa = tf.constant(2.0)
bb = tf.constant(4.0)
writer = tf.summary.FileWriter('.')
#writer.add_graph(tf.get_default_graph())
out = tf.random_uniform(shape=(3,))
vec1 = out + 1



In [7]:
var1 = tf.placeholder(tf.float32)
var2 = tf.placeholder(tf.float32)
z = var1 + var2
norm = tf.nn.l2_loss(z)

with tf.Session() as session:
    feed_dict={var1: [2,3], var2: [22,3]}
    answer, norm = session.run([z, norm], feed_dict=feed_dict)
    print(answer, norm)

[24.  6.] 306.0


In [4]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=4)
y = linear_model(x)
with tf.Session() as session:
    init = tf.global_variables_initializer()
    session.run(init)
    print(session.run(y, {x: [[1,2,3],[4,5,6]]}))

[[-3.6360192 -1.2333038 -1.2056456  2.4964776]
 [-8.733406  -2.9360428 -4.536804   5.6083126]]


In [19]:

from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [30]:


import os
X_train = tf.constant([[1],[2],[3],[4]], tf.float32)
y_train = tf.constant([[0], [-1], [-2], [-3]], tf.float32)
linear_model = tf.layers.Dense(units=1)
y_pred=linear_model(X_train)
losses = tf.losses.mean_squared_error(labels=y_train, predictions=y_pred)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(losses)
rank = tf.rank(X_train)
sumcol = X_train[2, :]
#print(X_train.eval())
print(X_train.shape)
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(300):
        what, loss_val = sess.run((train, losses))
        if(i % 100 == 0):
            print(loss_val)
            print(what)
        
    print(linear_model.name)
    print(sess.run(rank))
    print(sess.run(sumcol))

    #show_graph(tf.get_default_graph())
    #weights = tf.get_default_graph().get_tensor_by_name(linear_model.name + '/kernel')
    #print(weights)
        

(4, 1)
32.184044
None
0.24295047
None
0.1333769
None
dense_24
2
[3.]


In [38]:
#with tf.variable_scope("scope1"):
#    p = tf.get_variable("p", [1,2,3])
#with tf.variable_scope("scope2"):
#    p = tf.get_variable("p", [3,4,2])
with tf.Session() as sess:
    with tf.variable_scope("scope1"):
        r = sess.run(p, feed_dict={p: np.ones([1,2,3])})
        print(r)

ValueError: Cannot feed value of shape (1, 2, 3) for Tensor 'scope2/p:0', which has shape '(3, 4, 2)'